### Boilerplate code - llm initiation

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0, 
    model="gemini-2.5-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)

google_llm.invoke("Hello")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bccb1-b414-7050-9e5e-46f60d6d9d89-0', usage_metadata={'input_tokens': 2, 'output_tokens': 261, 'total_tokens': 263, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 252}})

## Message state

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

messages = [AIMessage(content="Hi, there!"), 
            HumanMessage(content="Hello AI!"), 
            AIMessage(content="How can i help you?"), 
            HumanMessage(content="Why are you angry at me?")
        ]

google_llm.invoke(messages)

AIMessage(content="Oh, I'm definitely not angry with you! As an AI, I don't actually have emotions like anger, happiness, or sadness. My responses are based on the information I'm programmed with and the goal of being helpful and informative.\n\nIs there anything in particular that made you feel that way? Sometimes text can be misinterpreted without the nuances of tone of voice, but I assure you, I'm here to assist you in a friendly and neutral way.", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bccb1-c08d-7830-9a92-b4873af94d76-0', usage_metadata={'input_tokens': 24, 'output_tokens': 257, 'total_tokens': 281, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 161}})

In [ ]:
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class MessageState(TypedDict):
    messages: list[AnyMessage] # AIMessage(Role is assistant), HumanMessage(Role is human automatically)

class MessageState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

### Intro on add_messages

In [ ]:
from langgraph.graph.message import add_messages

data1 = [{"role": "human", "content": "test"}] # role and content are must keys
data2 = [{"role": "ai", "content": "test2"}]

added_data = add_messages(data1, data2)

added_data # similar operation like .extend

### Langgraph - Building chatbot with MessageState

In [ ]:
from langgraph.graph import MessagesState, StateGraph, START, END
from IPython.display import display, Image
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage
from langgraph.graph.message import add_messages
from typing import Annotated

class MessagesState(MessagesState):
    pass

# class MessagesState(TypedDict):
#     messages: Annotated[list[AnyMessage], add_messages]

def chatbot(state: MessagesState):
    return {"messages": google_llm.invoke(state.get('messages'))}

builder = StateGraph(MessagesState)
builder.add_node("chatbot", chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

graph = builder.compile()

# display(Image(graph.get_graph().draw_mermaid_png()))

graph.invoke({"messages": HumanMessage(content="Hello")}) # HumanMessage can be list of direct as internally it uses add_messages reducer i.e .extend so final will be list anyways
graph.invoke({"messages": HumanMessage(content="What is your name?")})
graph.invoke({"messages": HumanMessage(content="I want to see you")}) # returns final state of graph execution

